In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/f1nalyze-datathon-ieeecsmuj/sample_submission.csv
/kaggle/input/f1nalyze-datathon-ieeecsmuj/validation.csv
/kaggle/input/f1nalyze-datathon-ieeecsmuj/train.csv
/kaggle/input/f1nalyze-datathon-ieeecsmuj/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

### 1. Importing Dataset

In [3]:
df = pd.read_csv('/kaggle/input/f1nalyze-datathon-ieeecsmuj/train.csv',encoding="utf-8")
valid_df = pd.read_csv("/kaggle/input/f1nalyze-datathon-ieeecsmuj/validation.csv",encoding="utf-8")
test_df = pd.read_csv("/kaggle/input/f1nalyze-datathon-ieeecsmuj/test.csv",encoding="utf-8")


/tmp/ipykernel_64/3742685509.py:1: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/f1nalyze-datathon-ieeecsmuj/train.csv',encoding="utf-8")
/tmp/ipykernel_64/3742685509.py:2: DtypeWarning: Columns (13,16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_df = pd.read_csv("/kaggle/input/f1nalyze-datathon-ieeecsmuj/validation.csv",encoding="utf-8")
/tmp/ipykernel_64/3742685509.py:3: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("/kaggle/input/f1nalyze-datathon-ieeecsmuj/test.csv",encoding="utf-8")


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
df.head()

,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,laps,time_x,timetaken_in_millisec,fastestLap,rank,fastestLapTime,max_speed,statusId,year,round,circuitId,grand_prix,date,time_y,url_x,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,driverRef,driver_num,driver_code,forename,surname,dob,nationality,url_y,driverStandingsId,raceId_y,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,20025,833,642,51,2,1,1,1,1,9.0,70,2:13:23.6,8003600,\N,\N,\N,\N,1,1950,1,9,British Grand Prix,1950-05-13,\N,http://en.wikipedia.org/wiki/1950_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,farina,\N,\N,Nino,Farina,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina,46079,792,3.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,922731975
1,20025,833,642,51,2,1,1,1,1,9.0,70,2:13:23.6,8003600,\N,\N,\N,\N,1,1950,1,9,British Grand Prix,1950-05-13,\N,http://en.wikipedia.org/wiki/1950_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,farina,\N,\N,Nino,Farina,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina,46101,793,6.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923172525
2,20025,833,642,51,2,1,1,1,1,9.0,70,2:13:23.6,8003600,\N,\N,\N,\N,1,1950,1,9,British Grand Prix,1950-05-13,\N,http://en.wikipedia.org/wiki/1950_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,farina,\N,\N,Nino,Farina,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina,46134,794,6.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923833350
3,20025,833,642,51,2,1,1,1,1,9.0,70,2:13:23.6,8003600,\N,\N,\N,\N,1,1950,1,9,British Grand Prix,1950-05-13,\N,http://en.wikipedia.org/wiki/1950_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,farina,\N,\N,Nino,Farina,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina,46202,795,10.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,925195050
4,20025,833,642,51,2,1,1,1,1,9.0,70,2:13:23.6,8003600,\N,\N,\N,\N,1,1950,1,9,British Grand Prix,1950-05-13,\N,http://en.wikipedia.org/wiki/1950_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,farina,\N,\N,Nino,Farina,1906-10-30,Italian,http://en.wikipedia.org/wiki/Nino_Farina,46271,796,10.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,926576775


In [6]:
valid_df.head()

,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,laps,time_x,timetaken_in_millisec,fastestLap,rank,fastestLapTime,max_speed,statusId,year,round,circuitId,grand_prix,date,time_y,url_x,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,driverRef,driver_num,driver_code,forename,surname,dob,nationality,url_y,driverStandingsId,raceId_y,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,22109,899,17,9,2,4,2,2,2,18.0,71,+10.452,5566752,51,1,1:15.436,205.636,1,2013,19,18,Brazilian Grand Prix,2013-11-24,16:00:00,http://en.wikipedia.org/wiki/2013_Brazilian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,webber,\N,WEB,Mark,Webber,1976-08-27,Australian,http://en.wikipedia.org/wiki/Mark_Webber_(raci...,63816,340,28.0,8,8,0,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1410907944
1,22109,899,17,9,2,4,2,2,2,18.0,71,+10.452,5566752,51,1,1:15.436,205.636,1,2013,19,18,Brazilian Grand Prix,2013-11-24,16:00:00,http://en.wikipedia.org/wiki/2013_Brazilian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,webber,\N,WEB,Mark,Webber,1976-08-27,Australian,http://en.wikipedia.org/wiki/Mark_Webber_(raci...,63840,341,53.0,4,4,1,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1411438560
2,22109,899,17,9,2,4,2,2,2,18.0,71,+10.452,5566752,51,1,1:15.436,205.636,1,2013,19,18,Brazilian Grand Prix,2013-11-24,16:00:00,http://en.wikipedia.org/wiki/2013_Brazilian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,webber,\N,WEB,Mark,Webber,1976-08-27,Australian,http://en.wikipedia.org/wiki/Mark_Webber_(raci...,63888,342,78.0,1,1,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1412499792
3,22109,899,17,9,2,4,2,2,2,18.0,71,+10.452,5566752,51,1,1:15.436,205.636,1,2013,19,18,Brazilian Grand Prix,2013-11-24,16:00:00,http://en.wikipedia.org/wiki/2013_Brazilian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,webber,\N,WEB,Mark,Webber,1976-08-27,Australian,http://en.wikipedia.org/wiki/Mark_Webber_(raci...,63912,343,93.0,1,1,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1413030408
4,22109,899,17,9,2,4,2,2,2,18.0,71,+10.452,5566752,51,1,1:15.436,205.636,1,2013,19,18,Brazilian Grand Prix,2013-11-24,16:00:00,http://en.wikipedia.org/wiki/2013_Brazilian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,webber,\N,WEB,Mark,Webber,1976-08-27,Australian,http://en.wikipedia.org/wiki/Mark_Webber_(raci...,63936,344,103.0,3,3,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1413561024


In [7]:
df.shape

(2830101, 55)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830101 entries, 0 to 2830100
Data columns (total 55 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   resultId                int64  
 1   racerId                 int64  
 2   driverId                int64  
 3   constructorId           int64  
 4   number                  object 
 5   grid                    int64  
 6   position_x              object 
 7   positionText_x          object 
 8   positionOrder           int64  
 9   points                  float64
 10  laps                    int64  
 11  time_x                  object 
 12  timetaken_in_millisec   object 
 13  fastestLap              object 
 14  rank                    object 
 15  fastestLapTime          object 
 16  max_speed               object 
 17  statusId                int64  
 18  year                    int64  
 19  round                   int64  
 20  circuitId               int64  
 21  grand_prix              object 

In [9]:
## Checking For null values
df.isnull().sum()

resultId                  0
racerId                   0
driverId                  0
constructorId             0
number                    0
grid                      0
position_x                0
positionText_x            0
positionOrder             0
points                    0
laps                      0
time_x                    0
timetaken_in_millisec     0
fastestLap                0
rank                      0
fastestLapTime            0
max_speed                 0
statusId                  0
year                      0
round                     0
circuitId                 0
grand_prix                0
date                      0
time_y                    0
url_x                     0
fp1_date                  0
fp1_time                  0
fp2_date                  0
fp2_time                  0
fp3_date                  0
fp3_time                  0
quali_date                0
quali_time                0
sprint_date               0
sprint_time               0
driverRef           

In [10]:
df.columns

Index(['resultId', 'racerId', 'driverId', 'constructorId', 'number', 'grid',
       'position_x', 'positionText_x', 'positionOrder', 'points', 'laps',
       'time_x', 'timetaken_in_millisec', 'fastestLap', 'rank',
       'fastestLapTime', 'max_speed', 'statusId', 'year', 'round', 'circuitId',
       'grand_prix', 'date', 'time_y', 'url_x', 'fp1_date', 'fp1_time',
       'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date',
       'quali_time', 'sprint_date', 'sprint_time', 'driverRef', 'driver_num',
       'driver_code', 'forename', 'surname', 'dob', 'nationality', 'url_y',
       'driverStandingsId', 'raceId_y', 'points_y', 'position',
       'positionText_y', 'wins', 'constructorRef', 'company', 'nationality_y',
       'url', 'status', 'result_driver_standing'],
      dtype='object')

In [99]:
imp_cols = ['resultId', 'racerId', 'driverId', 'constructorId', 'grid',
        'positionOrder', 'points', 'laps',
        'timetaken_in_millisec',
       'fastestLapTime', 'max_speed','round', 'circuitId',
       'grand_prix', 'date','forename', 'surname','nationality',
       'driverStandingsId','points_y','position',
        'wins','company', 'nationality_y',
        'status','result_driver_standing']

In [100]:
train_valid_df = valid_df[imp_cols]
train_df = df[imp_cols]

In [101]:
train_df.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,position,wins,company,nationality_y,status,result_driver_standing
0,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46079,3.33,3,0,Alfa Romeo,Swiss,Finished,922731975
1,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46101,6.33,3,0,Alfa Romeo,Swiss,Finished,923172525
2,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46134,6.33,4,0,Alfa Romeo,Swiss,Finished,923833350
3,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46202,10.33,3,0,Alfa Romeo,Swiss,Finished,925195050
4,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46271,10.33,4,0,Alfa Romeo,Swiss,Finished,926576775


In [102]:
result_test_df = test_df[['resultId', 'racerId', 'driverId', 'constructorId', 'grid',
        'positionOrder', 'points', 'laps',
        'timetaken_in_millisec',
       'fastestLapTime', 'max_speed','round', 'circuitId',
       'grand_prix', 'date','forename', 'surname','nationality',
       'driverStandingsId','points_y',
        'wins','company', 'nationality_y',
        'status','result_driver_standing']]

In [103]:
# train_valid_df = valid_df[['resultId', 'racerId', 'driverId', 'constructorId', 'grid',
#         'positionOrder', 'points', 'laps',
#         'timetaken_in_millisec',
#        'fastestLapTime', 'max_speed','round', 'circuitId',
#        'grand_prix', 'date','forename', 'surname','nationality',
#        'driverStandingsId','points_y', 'position',
#         'wins','company', 'nationality_y',
#         'status','result_driver_standing']]

In [104]:
# train_df = df[['resultId', 'racerId', 'driverId', 'constructorId', 'grid',
#         'positionOrder', 'points', 'laps',
#         'timetaken_in_millisec',
#        'fastestLapTime', 'max_speed','round', 'circuitId',
#        'grand_prix', 'date','forename', 'surname','nationality',
#        'driverStandingsId','points_y', 'position',
#         'wins','company', 'nationality_y',
#         'status','result_driver_standing']]

In [105]:
train_valid_df.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,position,wins,company,nationality_y,status,result_driver_standing
0,22109,899,17,9,4,2,18.0,71,5566752,1:15.436,205.636,19,18,Brazilian Grand Prix,2013-11-24,Mark,Webber,Australian,63816,28.0,8,0,Red Bull,Austrian,Finished,1410907944
1,22109,899,17,9,4,2,18.0,71,5566752,1:15.436,205.636,19,18,Brazilian Grand Prix,2013-11-24,Mark,Webber,Australian,63840,53.0,4,1,Red Bull,Austrian,Finished,1411438560
2,22109,899,17,9,4,2,18.0,71,5566752,1:15.436,205.636,19,18,Brazilian Grand Prix,2013-11-24,Mark,Webber,Australian,63888,78.0,1,2,Red Bull,Austrian,Finished,1412499792
3,22109,899,17,9,4,2,18.0,71,5566752,1:15.436,205.636,19,18,Brazilian Grand Prix,2013-11-24,Mark,Webber,Australian,63912,93.0,1,2,Red Bull,Austrian,Finished,1413030408
4,22109,899,17,9,4,2,18.0,71,5566752,1:15.436,205.636,19,18,Brazilian Grand Prix,2013-11-24,Mark,Webber,Australian,63936,103.0,3,2,Red Bull,Austrian,Finished,1413561024


In [106]:
## spliting lables and features
X_train = train_df.drop('position',axis=1)
y_train = train_df['position']


X_valid_train = train_valid_df.drop('position',axis=1)
y_valid_train = train_valid_df['position']

In [107]:
X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing
0,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46079,3.33,0,Alfa Romeo,Swiss,Finished,922731975
1,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46101,6.33,0,Alfa Romeo,Swiss,Finished,923172525
2,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46134,6.33,0,Alfa Romeo,Swiss,Finished,923833350
3,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46202,10.33,0,Alfa Romeo,Swiss,Finished,925195050
4,20025,833,642,51,1,1,9.0,70,8003600,\N,\N,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46271,10.33,0,Alfa Romeo,Swiss,Finished,926576775


In [109]:
X_train = X_train.replace("\\N", np.nan)


X_valid_train = X_valid_train.replace("\\N", np.nan)

result_test_df = result_test_df.replace("\\N",np.nan)

X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing
0,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46079,3.33,0,Alfa Romeo,Swiss,Finished,922731975
1,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46101,6.33,0,Alfa Romeo,Swiss,Finished,923172525
2,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46134,6.33,0,Alfa Romeo,Swiss,Finished,923833350
3,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46202,10.33,0,Alfa Romeo,Swiss,Finished,925195050
4,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46271,10.33,0,Alfa Romeo,Swiss,Finished,926576775


In [110]:
result_test_df.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing
0,23949,997,4,1,13,8,4.0,70,NaN,01:08.7,226.399,9,70,Austrian Grand Prix,01/07/18,Fernando,Alonso,Spanish,12474,16.0,0,McLaren,British,+1 Lap,298739826
1,23949,997,4,1,13,8,4.0,70,NaN,01:08.7,226.399,9,70,Austrian Grand Prix,01/07/18,Fernando,Alonso,Spanish,12494,21.0,0,McLaren,British,+1 Lap,299218806
2,23949,997,4,1,13,8,4.0,70,NaN,01:08.7,226.399,9,70,Austrian Grand Prix,01/07/18,Fernando,Alonso,Spanish,12514,21.0,0,McLaren,British,+1 Lap,299697786
3,23949,997,4,1,13,8,4.0,70,NaN,01:08.7,226.399,9,70,Austrian Grand Prix,01/07/18,Fernando,Alonso,Spanish,12534,25.0,0,McLaren,British,+1 Lap,300176766
4,23949,997,4,1,13,8,4.0,70,NaN,01:08.7,226.399,9,70,Austrian Grand Prix,01/07/18,Fernando,Alonso,Spanish,12554,25.0,0,McLaren,British,+1 Lap,300655746


### 2. Data Cleaning

In [111]:
X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing
0,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46079,3.33,0,Alfa Romeo,Swiss,Finished,922731975
1,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46101,6.33,0,Alfa Romeo,Swiss,Finished,923172525
2,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46134,6.33,0,Alfa Romeo,Swiss,Finished,923833350
3,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46202,10.33,0,Alfa Romeo,Swiss,Finished,925195050
4,20025,833,642,51,1,1,9.0,70,8003600,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46271,10.33,0,Alfa Romeo,Swiss,Finished,926576775


In [112]:
## Imputing NaN values with mean
imputer = SimpleImputer(strategy='mean')
X_train['timetaken_in_millisec'] = imputer.fit_transform(X_train[['timetaken_in_millisec']])
X_train['timetaken_in_millisec'] = X_train['timetaken_in_millisec'].astype('float64')



X_valid_train['timetaken_in_millisec'] = imputer.transform(X_valid_train[['timetaken_in_millisec']])
X_valid_train['timetaken_in_millisec'] = X_valid_train['timetaken_in_millisec'].astype('float64')


result_test_df['timetaken_in_millisec'] = imputer.transform(result_test_df[['timetaken_in_millisec']])
result_test_df['timetaken_in_millisec'] = result_test_df['timetaken_in_millisec'].astype('float64')

In [113]:
X_train['timetaken_in_millisec'].head()

0    8003600.0
1    8003600.0
2    8003600.0
3    8003600.0
4    8003600.0
Name: timetaken_in_millisec, dtype: float64

In [114]:
X_valid_train['timetaken_in_millisec'].head()

0    5566752.0
1    5566752.0
2    5566752.0
3    5566752.0
4    5566752.0
Name: timetaken_in_millisec, dtype: float64

In [115]:
X_train['timetaken_in_millisec'] = (X_train['timetaken_in_millisec']/60000)

X_valid_train['timetaken_in_millisec'] = (X_valid_train['timetaken_in_millisec']/60000)

result_test_df['timetaken_in_millisec'] = (result_test_df['timetaken_in_millisec']/60000)

In [116]:
X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing
0,20025,833,642,51,1,1,9.0,70,133.393333,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46079,3.33,0,Alfa Romeo,Swiss,Finished,922731975
1,20025,833,642,51,1,1,9.0,70,133.393333,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46101,6.33,0,Alfa Romeo,Swiss,Finished,923172525
2,20025,833,642,51,1,1,9.0,70,133.393333,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46134,6.33,0,Alfa Romeo,Swiss,Finished,923833350
3,20025,833,642,51,1,1,9.0,70,133.393333,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46202,10.33,0,Alfa Romeo,Swiss,Finished,925195050
4,20025,833,642,51,1,1,9.0,70,133.393333,NaN,NaN,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46271,10.33,0,Alfa Romeo,Swiss,Finished,926576775


In [117]:
X_train['fastestLapTime'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2830101 entries, 0 to 2830100
Series name: fastestLapTime
Non-Null Count   Dtype 
--------------   ----- 
718091 non-null  object
dtypes: object(1)
memory usage: 21.6+ MB


In [118]:
X_train['fastestLapTime'].value_counts()

fastestLapTime
1:14.117    779
1:16.203    725
1:35.877    685
1:33.133    680
1:25.728    661
           ... 
1:39.801     11
2:01.148     11
1:25.646     11
1:29.229     11
1:13.700     11
Name: count, Length: 3638, dtype: int64

In [119]:
def extract_minutes(time_string):
    if not pd.isna(time_string):
        minutes, seconds = time_string.split(':')
        minutes = float(minutes)
        seconds = float(seconds)/100
        total_time = minutes+seconds
        return total_time

X_train['fastestLapTime'] = X_train['fastestLapTime'].apply(extract_minutes)


X_valid_train['fastestLapTime'] = X_valid_train['fastestLapTime'].apply(extract_minutes)

result_test_df['fastestLapTime'] = result_test_df['fastestLapTime'].apply(extract_minutes)

X_train['fastestLapTime']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
2830096    1.15436
2830097    1.15436
2830098    1.15436
2830099    1.15436
2830100    1.15436
Name: fastestLapTime, Length: 2830101, dtype: float64

In [120]:
X_train['fastestLapTime'].value_counts()

fastestLapTime
1.14117    779
1.16203    725
1.35877    685
1.33133    680
1.25728    661
          ... 
1.39801     11
2.01148     11
1.25646     11
1.29229     11
1.13700     11
Name: count, Length: 3638, dtype: int64

In [121]:
X_train['fastestLapTime'] = imputer.fit_transform(X_train[['fastestLapTime']])
X_train['fastestLapTime'] = X_train['fastestLapTime'].astype('float64')
X_train['fastestLapTime']

X_valid_train['fastestLapTime'] = imputer.transform(X_valid_train[['fastestLapTime']])
X_valid_train['fastestLapTime'] = X_valid_train['fastestLapTime'].astype('float64')
X_valid_train['fastestLapTime']


result_test_df['fastestLapTime'] = imputer.transform(result_test_df[['fastestLapTime']])
result_test_df['fastestLapTime'] = result_test_df['fastestLapTime'].astype('float64')

In [122]:
X_train['max_speed'].value_counts()

max_speed
206.483    674
202.262    642
226.600    635
250.162    628
196.523    621
          ... 
219.825     11
244.794     11
207.296     11
208.369     11
216.468     11
Name: count, Length: 3720, dtype: int64

In [123]:
X_train['max_speed'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2830101 entries, 0 to 2830100
Series name: max_speed
Non-Null Count   Dtype 
--------------   ----- 
718091 non-null  object
dtypes: object(1)
memory usage: 21.6+ MB


In [124]:
most_frequent_imputer = SimpleImputer(strategy = 'most_frequent')
X_train["max_speed"] = pd.DataFrame(most_frequent_imputer.fit_transform(X_train[["max_speed"]]),columns=["max_speed"],index=X_train.index).astype(np.float64)


X_valid_train["max_speed"] = pd.DataFrame(most_frequent_imputer.transform(X_valid_train[["max_speed"]]),columns=["max_speed"],index=X_valid_train.index).astype(np.float64)


result_test_df["max_speed"] = pd.DataFrame(most_frequent_imputer.transform(result_test_df[["max_speed"]]),columns=["max_speed"],index=result_test_df.index).astype(np.float64)

In [125]:
X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing
0,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46079,3.33,0,Alfa Romeo,Swiss,Finished,922731975
1,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46101,6.33,0,Alfa Romeo,Swiss,Finished,923172525
2,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46134,6.33,0,Alfa Romeo,Swiss,Finished,923833350
3,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46202,10.33,0,Alfa Romeo,Swiss,Finished,925195050
4,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,British Grand Prix,1950-05-13,Nino,Farina,Italian,46271,10.33,0,Alfa Romeo,Swiss,Finished,926576775


In [126]:
 X_train["max_speed"].value_counts()

max_speed
206.483    2112684
220.611        753
202.262        642
226.600        635
250.162        628
            ...   
215.364         11
210.480         11
244.794         11
207.296         11
208.369         11
Name: count, Length: 3700, dtype: int64

In [127]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830101 entries, 0 to 2830100
Data columns (total 25 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   resultId                int64  
 1   racerId                 int64  
 2   driverId                int64  
 3   constructorId           int64  
 4   grid                    int64  
 5   positionOrder           int64  
 6   points                  float64
 7   laps                    int64  
 8   timetaken_in_millisec   float64
 9   fastestLapTime          float64
 10  max_speed               float64
 11  round                   int64  
 12  circuitId               int64  
 13  grand_prix              object 
 14  date                    object 
 15  forename                object 
 16  surname                 object 
 17  nationality             object 
 18  driverStandingsId       int64  
 19  points_y                float64
 20  wins                    int64  
 21  company                 object 

In [128]:
df["grand_prix"].value_counts()

grand_prix
British Grand Prix               188782
Italian Grand Prix               186126
Monaco Grand Prix                181741
German Grand Prix                178167
Belgian Grand Prix               168052
French Grand Prix                162820
Canadian Grand Prix              152180
Brazilian Grand Prix             145095
Spanish Grand Prix               143483
Australian Grand Prix            110964
Japanese Grand Prix              108150
Hungarian Grand Prix             107257
United States Grand Prix         101514
San Marino Grand Prix             95878
Austrian Grand Prix               83916
European Grand Prix               83828
Dutch Grand Prix                  69052
South African Grand Prix          63516
Malaysian Grand Prix              59715
Portuguese Grand Prix             52544
Argentine Grand Prix              45994
Mexican Grand Prix                41069
Chinese Grand Prix                40779
Bahrain Grand Prix                36728
Turkish Grand Prix           

In [129]:
le = LabelEncoder()
X_train['grand_prix'] = le.fit_transform(X_train['grand_prix'])

X_valid_train['grand_prix'] = le.fit_transform(X_valid_train['grand_prix'])

result_test_df['grand_prix'] = le.fit_transform(result_test_df['grand_prix'])


In [130]:
X_train['grand_prix'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2830101 entries, 0 to 2830100
Series name: grand_prix
Non-Null Count    Dtype
--------------    -----
2830101 non-null  int64
dtypes: int64(1)
memory usage: 21.6 MB


In [133]:
X_train['date'] = pd.to_datetime(X_train['date'])

X_valid_train['date'] = pd.to_datetime(X_valid_train['date'])

result_test_df['date'] = pd.to_datetime(result_test_df['date'])

X_train['date']

0         1950-05-13
1         1950-05-13
2         1950-05-13
3         1950-05-13
4         1950-05-13
             ...    
2830096   2013-11-24
2830097   2013-11-24
2830098   2013-11-24
2830099   2013-11-24
2830100   2013-11-24
Name: date, Length: 2830101, dtype: datetime64[ns]

In [137]:
X_train['year'] = (X_train['date'].dt.year).astype(np.int64)
X_train['month'] = (X_train['date'].dt.month).astype(np.int64)
X_train['day'] = (X_train['date'].dt.day).astype(np.int64)


X_valid_train['year'] = (X_valid_train['date'].dt.year).astype(np.int64)
X_valid_train['month'] = (X_valid_train['date'].dt.month).astype(np.int64)
X_valid_train['day'] = (X_valid_train['date'].dt.day).astype(np.int64)


result_test_df['year'] = (result_test_df['date'].dt.year).astype(np.int64)
result_test_df['month'] = (result_test_df['date'].dt.month).astype(np.int64)
result_test_df['day'] = (result_test_df['date'].dt.day).astype(np.int64)

In [139]:
X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,forename,surname,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing,year,month,day
0,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,Nino,Farina,Italian,46079,3.33,0,Alfa Romeo,Swiss,Finished,922731975,1950,5,13
1,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,Nino,Farina,Italian,46101,6.33,0,Alfa Romeo,Swiss,Finished,923172525,1950,5,13
2,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,Nino,Farina,Italian,46134,6.33,0,Alfa Romeo,Swiss,Finished,923833350,1950,5,13
3,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,Nino,Farina,Italian,46202,10.33,0,Alfa Romeo,Swiss,Finished,925195050,1950,5,13
4,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,Nino,Farina,Italian,46271,10.33,0,Alfa Romeo,Swiss,Finished,926576775,1950,5,13


In [140]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830101 entries, 0 to 2830100
Data columns (total 28 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   resultId                int64         
 1   racerId                 int64         
 2   driverId                int64         
 3   constructorId           int64         
 4   grid                    int64         
 5   positionOrder           int64         
 6   points                  float64       
 7   laps                    int64         
 8   timetaken_in_millisec   float64       
 9   fastestLapTime          float64       
 10  max_speed               float64       
 11  round                   int64         
 12  circuitId               int64         
 13  grand_prix              int64         
 14  date                    datetime64[ns]
 15  forename                object        
 16  surname                 object        
 17  nationality             object        
 18  dr

In [141]:
# ord_encoder = OrdinalEncoder()
# X_train['date'] = ord_encoder.fit_transform(X_train[['date']]).astype(np.float64)



# X_valid_train['date'] = ord_encoder.fit_transform(X_valid_train[['date']]).astype(np.float64)
# X_valid_train['date'].value_counts()


# result_test_df['date'] = ord_encoder.fit_transform(result_test_df[['date']]).astype(np.float64)

In [143]:
X_train['full_name'] = X_train['forename'] + ' ' + X_train['surname']
X_train.drop(columns=['forename','surname'], axis=1, inplace=True)


X_valid_train['full_name'] = X_valid_train['forename'] + ' ' + X_valid_train['surname']
X_valid_train.drop(columns=['forename','surname'], axis=1, inplace=True)


result_test_df['full_name'] = result_test_df['forename'] + ' ' + result_test_df['surname']
result_test_df.drop(columns=['forename','surname'], axis=1, inplace=True)


In [144]:
X_train['full_name'] = le.fit_transform(X_train['full_name'])
X_train['nationality'] = le.fit_transform(X_train['nationality'])

In [145]:
X_valid_train['nationality'] = le.fit_transform(X_valid_train['nationality'])
X_valid_train['full_name'] = le.fit_transform(X_valid_train['full_name'])


In [146]:
result_test_df['nationality'] = le.fit_transform(result_test_df['nationality'])
result_test_df['full_name'] = le.fit_transform(result_test_df['full_name'])

In [147]:
X_train.head()


,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,timetaken_in_millisec,fastestLapTime,max_speed,round,circuitId,grand_prix,date,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing,year,month,day,full_name
0,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46079,3.33,0,Alfa Romeo,Swiss,Finished,922731975,1950,5,13,590
1,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46101,6.33,0,Alfa Romeo,Swiss,Finished,923172525,1950,5,13,590
2,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46134,6.33,0,Alfa Romeo,Swiss,Finished,923833350,1950,5,13,590
3,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46202,10.33,0,Alfa Romeo,Swiss,Finished,925195050,1950,5,13,590
4,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46271,10.33,0,Alfa Romeo,Swiss,Finished,926576775,1950,5,13,590


In [148]:
X_train['company'] = le.fit_transform(X_train['company'])
X_train['nationality_y'] = le.fit_transform(X_train['nationality_y'])
X_train['status'] = le.fit_transform(X_train['status'])



X_valid_train['company'] = le.fit_transform(X_valid_train['company'])
X_valid_train['nationality_y'] = le.fit_transform(X_valid_train['nationality_y'])
X_valid_train['status'] = le.fit_transform(X_valid_train['status'])

X_valid_train.head()


result_test_df['company'] = le.fit_transform(result_test_df['company'])
result_test_df['nationality_y'] = le.fit_transform(result_test_df['nationality_y'])
result_test_df['status'] = le.fit_transform(result_test_df['status'])

In [149]:
X_train = X_train.rename(columns={'timetaken_in_millisec': 'time_taken'})

X_valid_train = X_valid_train.rename(columns={'timetaken_in_millisec': 'time_taken'})

result_test_df = result_test_df.rename(columns={'timetaken_in_millisec': 'time_taken'})


In [150]:
X_train['grid'].value_counts()

grid
1     170021
2     164833
3     164320
4     160292
5     157570
6     148051
7     140700
8     139465
9     134352
10    126465
11    124158
12    119157
13    112145
14    111865
15    101993
0      99328
16     98634
17     90927
18     87099
19     79304
20     75137
21     58198
22     50759
23     35156
24     33389
25     23324
26     17078
27      2120
28      1266
29       888
31       654
32       596
30       553
33       301
34         3
Name: count, dtype: int64

In [151]:
X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,time_taken,fastestLapTime,max_speed,round,circuitId,grand_prix,date,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing,year,month,day,full_name
0,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46079,3.33,0,4,23,62,922731975,1950,5,13,590
1,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46101,6.33,0,4,23,62,923172525,1950,5,13,590
2,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46134,6.33,0,4,23,62,923833350,1950,5,13,590
3,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46202,10.33,0,4,23,62,925195050,1950,5,13,590
4,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46271,10.33,0,4,23,62,926576775,1950,5,13,590


In [152]:
X_train['wins'].value_counts()

wins
0     2245394
1      274899
2      125623
3       63631
4       40952
5       27558
6       22382
7       10689
8        6527
9        5486
10       3008
11       1864
12       1352
13        736
Name: count, dtype: int64

In [154]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830101 entries, 0 to 2830100
Data columns (total 27 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   resultId                int64         
 1   racerId                 int64         
 2   driverId                int64         
 3   constructorId           int64         
 4   grid                    int64         
 5   positionOrder           int64         
 6   points                  float64       
 7   laps                    int64         
 8   time_taken              float64       
 9   fastestLapTime          float64       
 10  max_speed               float64       
 11  round                   int64         
 12  circuitId               int64         
 13  grand_prix              int64         
 14  date                    datetime64[ns]
 15  nationality             int64         
 16  driverStandingsId       int64         
 17  points_y                float64       
 18  wi

In [155]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# columns_to_scale = ['points','laps','time_taken','fastestLapTime','max_speed','points_y']

# X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
# X_valid_train[columns_to_scale] = scaler.fit_transform(X_valid_train[columns_to_scale])

In [156]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(X_train)
# X_train_scaled = pd.DataFrame(scaled_data, columns=X_train.columns)

In [157]:
# scaled_data = scaler.fit_transform(X_valid_train)
# X_valid_train_scaled = pd.DataFrame(scaled_data, columns=X_valid_train.columns)

In [158]:
duplicate_labels = df.index[df.index.duplicated()]
duplicate_labels

Index([], dtype='int64')

In [159]:
X_valid_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,time_taken,fastestLapTime,max_speed,round,circuitId,grand_prix,date,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing,year,month,day,full_name
0,22109,899,17,9,4,2,18.0,71,92.7792,1.15436,205.636,19,18,6,2013-11-24,1,63816,28.0,0,9,1,22,1410907944,2013,11,24,27
1,22109,899,17,9,4,2,18.0,71,92.7792,1.15436,205.636,19,18,6,2013-11-24,1,63840,53.0,1,9,1,22,1411438560,2013,11,24,27
2,22109,899,17,9,4,2,18.0,71,92.7792,1.15436,205.636,19,18,6,2013-11-24,1,63888,78.0,2,9,1,22,1412499792,2013,11,24,27
3,22109,899,17,9,4,2,18.0,71,92.7792,1.15436,205.636,19,18,6,2013-11-24,1,63912,93.0,2,9,1,22,1413030408,2013,11,24,27
4,22109,899,17,9,4,2,18.0,71,92.7792,1.15436,205.636,19,18,6,2013-11-24,1,63936,103.0,2,9,1,22,1413561024,2013,11,24,27


In [160]:
X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,time_taken,fastestLapTime,max_speed,round,circuitId,grand_prix,date,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing,year,month,day,full_name
0,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46079,3.33,0,4,23,62,922731975,1950,5,13,590
1,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46101,6.33,0,4,23,62,923172525,1950,5,13,590
2,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46134,6.33,0,4,23,62,923833350,1950,5,13,590
3,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46202,10.33,0,4,23,62,925195050,1950,5,13,590
4,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46271,10.33,0,4,23,62,926576775,1950,5,13,590


In [161]:
nan_columns = X_train.isna().any()
nan_columns

resultId                  False
racerId                   False
driverId                  False
constructorId             False
grid                      False
positionOrder             False
points                    False
laps                      False
time_taken                False
fastestLapTime            False
max_speed                 False
round                     False
circuitId                 False
grand_prix                False
date                      False
nationality               False
driverStandingsId         False
points_y                  False
wins                      False
company                   False
nationality_y             False
status                    False
result_driver_standing    False
year                      False
month                     False
day                       False
full_name                 False
dtype: bool

In [162]:
y_train.value_counts()

position
1      169955
2      164339
3      154148
4      152231
5      144894
        ...  
103         4
105         3
106         3
107         1
108         1
Name: count, Length: 108, dtype: int64

In [163]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [164]:
X_train.head()

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,time_taken,fastestLapTime,max_speed,round,circuitId,grand_prix,date,nationality,driverStandingsId,points_y,wins,company,nationality_y,status,result_driver_standing,year,month,day,full_name
0,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46079,3.33,0,4,23,62,922731975,1950,5,13,590
1,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46101,6.33,0,4,23,62,923172525,1950,5,13,590
2,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46134,6.33,0,4,23,62,923833350,1950,5,13,590
3,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46202,10.33,0,4,23,62,925195050,1950,5,13,590
4,20025,833,642,51,1,1,9.0,70,133.393333,1.31119,206.483,1,9,7,1950-05-13,22,46271,10.33,0,4,23,62,926576775,1950,5,13,590


In [165]:
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42,max_features=1.0)
gbr.fit(X_train.drop(columns=['date']), y_train)

GradientBoostingRegressor(max_depth=4, max_features=1.0, random_state=42)

In [168]:
y_pred_train = gbr.predict(X_valid_train.drop(columns=['date']))
mse = mean_squared_error(y_valid_train, y_pred_train)
rmse = np.sqrt(mse)

In [169]:
mse

16.220125111211185

In [170]:
rmse

4.027421645570673

In [171]:
y_pred_test = gbr.predict(result_test_df.drop(columns=['date']))

In [172]:
y_pred_test[:20]

array([ 6.39342255,  6.43345409,  6.94831763,  6.07754545,  6.07754545,
        6.07754545,  5.9795299 ,  5.9795299 ,  5.91304939,  5.91304939,
        5.91304939,  5.91304939,  5.85237237,  5.79675891,  5.79675891,
       11.4704236 ,  7.78791125,  6.78682521,  6.42604011,  5.27403538])

In [173]:
output = pd.DataFrame(
{
    "position":y_pred_test,
    "result_driver_standing":result_test_df['result_driver_standing']
})

In [174]:
output.head()

,position,result_driver_standing
0,6.393423,298739826
1,6.433454,299218806
2,6.948318,299697786
3,6.077545,300176766
4,6.077545,300655746


In [175]:
output.to_csv("submission.csv",index= False)

In [176]:
output.shape

(352928, 2)